#### 1. Доработать приложение по поиску авиабилетов, чтобы оно возвращало билеты по названию города, а не по IATA коду. (У aviasales есть для этого дополнительное API) Пункт отправления и пункт назначения должны передаваться в качестве параметров. Сделать форматированный вывод, который содержит в себе пункт отправления, пункт назначения, дату вылета, цену билета (можно добавить еще другие параметры по желанию).

In [1]:
import pandas as pd
import requests
import json
from pprint import pprint

In [2]:
# # %load aviasales.py
# from pprint import pprint
# import requests
# import json
# service = 'http://min-prices.aviasales.ru/calendar_preload?'
# fromCity = 'MOW'
# toCity = 'LED'
# link = f'{service}origin={fromCity}&destination={toCity}&one_way=true'
# req = requests.get(link)
# data = json.loads(req.text)
# for i in data['best_prices']:
#     print(i['value'],i['depart_date'],i['return_date'],i['gate'])


In [3]:
def get_cities():
    
    link = 'http://api.travelpayouts.com/data/ru/cities.json'
    
    with requests.session() as s:
        r = s.get(link)
        
    data = json.loads(r.text)
    
    cities = dict()
    
    for item in data:
        cities[item['name']] = item['code']
        cities[item['name_translations']['en']] = item['code']
        
    return cities

    
def get_tickets(origin_name, destination_name, one_way_ticket=True):
    
    cities = get_cities()

    origin = cities.get(origin_name, '--')
    destination = cities.get(destination_name, '--')
    
    ticket_type = {True: 'true', False: 'false'}
    
    one_way = ticket_type.get(one_way_ticket, True)
    
    service = 'http://min-prices.aviasales.ru/calendar_preload?'

    link = f'{service}origin={origin}&destination={destination}&one_way={one_way}'

    with requests.session() as s:
        r = s.get(link)

    data = json.loads(r.text)
    
    feature_names = ['origin', 'destination', 'gate', 'depart_date', 'return_date', 'number_of_changes', 'value']
    
    tickets = pd.DataFrame(data['best_prices'])[feature_names]
    tickets['origin'] = origin_name
    tickets['destination'] = destination_name
    
    for idx, row in tickets.iterrows():
    
        print('*' * 42)
        print(f'Вариант №:             {idx+1}')
        print(f'Пункт отправления:     {row[0]}')
        print(f'Пункт назначения:      {row[1]}')
        print(f'Продавец:              {row[2]}')
        print(f'Дата отправления:      {row[3]}')
        print(f'Дата возвращения:      {row[4]}')
        print(f'Количество пересадок:  {row[5]}')
        print(f'Цена билета:           {row[6]}')
        print('*' * 42)
        print('\n')

    return tickets


In [4]:
tickets = get_tickets('Москва', 'Волгоград', one_way_ticket=False)

******************************************
Вариант №:             1
Пункт отправления:     Москва
Пункт назначения:      Волгоград
Продавец:              Pobeda
Дата отправления:      2019-08-26
Дата возвращения:      2019-09-10
Количество пересадок:  0
Цена билета:           6200.0
******************************************


******************************************
Вариант №:             2
Пункт отправления:     Москва
Пункт назначения:      Волгоград
Продавец:              Nordwind
Дата отправления:      2019-09-06
Дата возвращения:      2019-09-21
Количество пересадок:  0
Цена билета:           7054.0
******************************************


******************************************
Вариант №:             3
Пункт отправления:     Москва
Пункт назначения:      Волгоград
Продавец:              Pobeda
Дата отправления:      2020-01-17
Дата возвращения:      2020-01-19
Количество пересадок:  0
Цена билета:           4600.0
******************************************


***********

Дата возвращения:      2019-12-12
Количество пересадок:  0
Цена билета:           6099.0
******************************************


******************************************
Вариант №:             135
Пункт отправления:     Москва
Пункт назначения:      Волгоград
Продавец:              Pobeda
Дата отправления:      2019-10-11
Дата возвращения:      2019-10-14
Количество пересадок:  0
Цена билета:           5400.0
******************************************


******************************************
Вариант №:             136
Пункт отправления:     Москва
Пункт назначения:      Волгоград
Продавец:              Pobeda
Дата отправления:      2020-01-01
Дата возвращения:      2020-01-04
Количество пересадок:  0
Цена билета:           6800.0
******************************************


******************************************
Вариант №:             137
Пункт отправления:     Москва
Пункт назначения:      Волгоград
Продавец:              Pobeda
Дата отправления:      2020-03-12
Дата во

In [5]:
tickets.head(2)

,origin,destination,gate,depart_date,return_date,number_of_changes,value
0,Москва,Волгоград,Pobeda,2019-08-26,2019-09-10,0,6200.0
1,Москва,Волгоград,Nordwind,2019-09-06,2019-09-21,0,7054.0


#### 2. В приложении парсинга википедии получить первую ссылку из раздела "Ссылки" и вывести все значимые слова из неё. Результат записать в файл в форматированном виде.

In [6]:
import re
from collections import Counter

In [7]:
# # %load wiki.py
# from pprint import pprint
# import requests
# import re

# def get_link(topic):
#     link='https://ru.wikipedia.org/wiki/'+topic.capitalize()
#     return link

# def get_topic_page(topic):
#     link = get_link(topic)
#     html = requests.get(link).text
#     return html

# def get_topic_text(topic):
#     html_content = get_topic_page(topic)
#     words = re.findall("[а-яА-Я]{3,}",html_content)
#     #text = ' '.join(words)
#     return words

# # text = get_topic_text('Дерево')
# # print(len(text))
# # print(text[0:1000])

# def get_common_words(topic):
#     words_list = get_topic_text(topic)
#     rate={}
#     for word in words_list:
#         if word in rate:
#             rate[word]+=1
#         else:
#             rate[word]=1
#     rate_list = list(rate.items())
#     rate_list.sort(key = lambda x: -x[1])
#     return rate_list

# dict1 = get_common_words('Россия')
# pprint(dict1)


In [17]:
def get_link(topic):
    link='https://ru.wikipedia.org/wiki/'+topic.capitalize()
    return link

def get_topic_page(topic):
    link = get_link(topic)
    html = requests.get(link).text
    return html

def get_first_linked_page(topic, n_words=10):
    html_content = get_topic_page(topic)
    try:
        text = re.findall(r'<li><a rel=.+\sclass=.+\shref=\"https?://\S+?\">', html_content)
        for idx, line in enumerate(text[:1]):
            link = re.findall(r'"((http)s?://.*?)"', line)[0][0]
            try:
                html = requests.get(link, timeout=5).text
                words = re.findall(r'[а-яА-Я]{3,}', html)
                counter = Counter(words)
                most_common_words = dict(counter.most_common(n_words))
                file_name = topic.capitalize() + '_' + '0' * (4 - len(str(idx))) + str(idx+1) + '.json'
                with open(file_name, 'w', encoding='utf-8') as f:
                    f.write(link)
                with open(file_name, 'a', encoding='utf-8') as f:
                    json.dump(most_common_words, f, ensure_ascii=False)
                print(f'Ссылка №{idx+1}: {link}')
                print(f'{n_words} значимых слов:')
                pprint(most_common_words)
                print(f'Data saved to: {file_name}')
                print('\n')
            except:
                continue
    except:
        pass

In [18]:
get_first_linked_page('Москва')

Ссылка №1: https://www.mos.ru/
10 значимых слов:
{'Москве': 5,
 'Москвы': 10,
 'Московское': 5,
 'Мэра': 5,
 'Официальный': 5,
 'города': 4,
 'для': 6,
 'долголетие': 5,
 'как': 4,
 'сайт': 5}
Data saved to: Москва_0001.json




#### 3.* Научить приложение определять количество ссылок в статье (раздел Ссылки). Выполнить поиск слов в статьях по каждой ссылке и результаты записать в отдельные файлы.

In [15]:
def get_all_linked_pages(topic, n_words=10):
    html_content = get_topic_page(topic)
    try:
        text = re.findall(r'<li><a rel=.+\sclass=.+\shref=\"https?://\S+?\">', html_content)
        print(f'Кол-во ссылок в статье по теме {topic}: {len(text)}')
        print('\n')
        for idx, line in enumerate(text):
            link = re.findall(r'"((http)s?://.*?)"', line)[0][0]
            try:
                html = requests.get(link, timeout=5).text
                words = re.findall(r'[а-яА-Я]{3,}', html)
                counter = Counter(words)
                most_common_words = dict(counter.most_common(n_words))
                file_name = topic.capitalize() + '_' + '0' * (4 - len(str(idx)))+ str(idx+1) + '.json'
                with open(file_name, 'w', encoding='utf-8') as f:
                    f.write(link)
                with open(file_name, 'a', encoding='utf-8') as f:
                    json.dump(most_common_words, f, ensure_ascii=False)
                print(f'Ссылка №{idx+1}: {link}')
                print(f'{n_words} значимых слов:')
                pprint(most_common_words)
                print(f'Data saved to: {file_name}')
                print('\n')
            except:
                continue
    except:
        pass

In [16]:
get_all_linked_pages('Москва')

Кол-во ссылок в статье по теме Москва: 6


Ссылка №1: https://www.mos.ru/
10 значимых слов:
{'Москве': 5,
 'Москвы': 10,
 'Московское': 5,
 'Мэра': 5,
 'Официальный': 5,
 'города': 4,
 'для': 6,
 'долголетие': 5,
 'как': 4,
 'сайт': 5}
Data saved to: Москва_0001.json


Ссылка №2: https://um.mos.ru
10 значимых слов:
{'Войти': 10,
 'Зарядье': 14,
 'Москвы': 9,
 'Нескучное': 14,
 'Прогулка': 8,
 'Садовые': 14,
 'дом': 11,
 'мосты': 14,
 'парке': 14,
 'усадьбы': 14}
Data saved to: Москва_0002.json


Ссылка №3: http://moscow.gramota.ru/
10 значимых слов:
{}
Data saved to: Москва_0003.json


Ссылка №5: http://mosday.ru/photos/gallery.php?base=1+3+5&amp;size=2&amp;frames=0&amp;alt=9&amp;rows=many
10 значимых слов:
{'адрес': 8,
 'все': 10,
 'выбрать': 47,
 'дата': 5,
 'добавить': 34,
 'название': 8,
 'пояснение': 6,
 'район': 4,
 'страница': 6,
 'съемки': 6}
Data saved to: Москва_0005.json


Ссылка №6: https://book-olds.ru/BookLibrary/24400-Vsya-Moskva.html
10 значимых слов:
{'Адрес': 29,
 'Мо